### Dask client + PyTorch

In [ ]:
from dask.distributed import Client, fire_and_forget

client = Client(memory_limit='5GB', n_workers=2, processes=True, dashboard_address=':8791')
display(client)

In [ ]:
import time
import numpy as np
import torch
N = 1000

class A:
    def __init__(self):
        self.x = torch.rand(N, N)

class Kernel:
    def __init__(self):
        self.count = 0
        
    def kernel(self, obj):
        self.count += 1
        y = torch.rand(N, N)
        for _ in range(10):
            y = y + torch.rand_like(y)
        # return torch.matmul(obj.x, y)  # this cannot be parallelized by Dask client.
        return y

op = Kernel()
for _ in range(500):
    obj = A()
    obj_ = client.scatter(obj)
    future = client.submit(op.kernel, obj_)
    fire_and_forget(future)

# op.kernel(A())
# op.count